<a href="https://colab.research.google.com/github/sophielu05/114-1-/blob/main/HW5_%E5%8D%88%E9%A4%90_%E8%81%9A%E6%9C%83%E6%B1%BA%E7%AD%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-search-results folium geopy -q

  Preparing metadata (setup.py) ... done


In [ ]:
from serpapi import GoogleSearch
import folium
from geopy.geocoders import Nominatim
import json
import re
from datetime import datetime
from IPython.display import display

In [ ]:
SERPAPI_KEY = "9a640622fc27d7b63ab8fc7ce7d375ff1758c781effa32ee12b0bb5822834d9a"

## 4. 定義搜尋與地理位置函數

In [ ]:
# 初始化地理編碼器
geolocator = Nominatim(user_agent="search_map_app")


TAIWAN_LOCATIONS = [
    # 直轄市
    '台北', '台北市', '臺北', '臺北市', 'Taipei',
    '新北', '新北市', 'New Taipei',
    '桃園', '桃園市', 'Taoyuan',
    '台中', '台中市', '臺中', '臺中市', 'Taichung',
    '台南', '台南市', '臺南', '臺南市', 'Tainan',
    '高雄', '高雄市', 'Kaohsiung',

    # 縣市
    '基隆', '基隆市', 'Keelung',
    '新竹', '新竹市', '新竹縣', 'Hsinchu',
    '苗栗', '苗栗縣', 'Miaoli',
    '彰化', '彰化縣', 'Changhua',
    '南投', '南投縣', 'Nantou',
    '雲林', '雲林縣', 'Yunlin',
    '嘉義', '嘉義市', '嘉義縣', 'Chiayi',
    '屏東', '屏東縣', 'Pingtung',
    '宜蘭', '宜蘭縣', 'Yilan',
    '花蓮', '花蓮縣', 'Hualien',
    '台東', '台東縣', '臺東', '臺東縣', 'Taitung',
    '澎湖', '澎湖縣', 'Penghu',
    '金門', '金門縣', 'Kinmen',
    '連江', '連江縣', '馬祖', 'Matsu',

    # 知名景點/地標
    '101', '台北101', '西門町', '信義區', '中正區',
    '士林', '北投', '淡水', '九份', '平溪',
    '日月潭', '阿里山', '墾丁', '太魯閣',
    '東大門', '逢甲', '一中街', '勤美',
]

def search_google(query, api_key, num=5, gl='tw', hl='zh-tw'):
    """
    使用 SerpAPI 進行 Google 搜尋
    """
    try:
        params = {
            'q': query,
            'api_key': api_key,
            'num': num,
            'gl': gl,
            'hl': hl
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        return results.get('organic_results', [])

    except Exception as e:
        print(f"搜尋時發生錯誤: {e}")
        return []


def extract_locations_from_text(text):
    """
    從文字中擷取地區名稱

    參數:
        text (str): 要分析的文字

    回傳:
        list: 找到的地區名稱列表
    """
    found_locations = []
    text_lower = text.lower()

    for location in TAIWAN_LOCATIONS:
        if location.lower() in text_lower:
            # 避免重複加入相似地名
            base_name = location.replace('市', '').replace('縣', '')
            if base_name not in [loc.replace('市', '').replace('縣', '') for loc in found_locations]:
                found_locations.append(location)

    return found_locations


def get_coordinates(location_name):
    """
    取得地點的經緯度座標

    參數:
        location_name (str): 地點名稱

    回傳:
        tuple: (緯度, 經度) 或 None
    """
    try:
        # 加上台灣以提高準確度
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        print(f"無法取得 {location_name} 的座標: {e}")
        return None


def analyze_results_with_locations(results):
    """
    分析搜尋結果並擷取地理位置資訊

    參數:
        results (list): 搜尋結果列表

    回傳:
        list: 包含位置資訊的結果字典列表
    """
    analyzed_results = []

    for result in results:
        # 從標題和描述中尋找地點
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        combined_text = f"{title} {snippet}"

        locations = extract_locations_from_text(combined_text)

        if locations:
            analyzed_results.append({
                'title': title,
                'snippet': snippet,
                'link': result.get('link', ''),
                'locations': locations
            })

    return analyzed_results


def create_map_from_results(analyzed_results, center=[23.5, 121.0], zoom=7):
    """
    根據分析結果建立地圖

    參數:
        analyzed_results (list): 包含位置資訊的結果列表
        center (list): 地圖中心座標 [緯度, 經度]
        zoom (int): 地圖縮放層級

    回傳:
        folium.Map: 地圖物件
    """
    # 建立地圖
    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')

    # 用於記錄已標註的位置，避免重複
    marked_locations = {}

    # 為每個結果加上標記
    for result in analyzed_results:
        for location in result['locations']:
            coords = get_coordinates(location)

            if coords:
                lat, lon = coords

                # 如果該位置還沒標記過
                if location not in marked_locations:
                    marked_locations[location] = []

                # 將結果加到該位置
                marked_locations[location].append(result)

    # 在地圖上標註
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
              'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue']

    for i, (location, results) in enumerate(marked_locations.items()):
        coords = get_coordinates(location)
        if coords:
            lat, lon = coords

            # 建立 popup 內容
            popup_html = f"<div style='width: 300px;'>"
            popup_html += f"<h4 style='color: #2c3e50;'>{location}</h4>"
            popup_html += f"<p style='color: #7f8c8d; font-size: 12px;'>找到 {len(results)} 筆相關結果</p>"

            for j, res in enumerate(results, 1):
                popup_html += f"<div style='margin: 10px 0; padding: 10px; background: #ecf0f1; border-radius: 5px;'>"
                popup_html += f"<b style='color: #2980b9;'>[{j}] {res['title'][:50]}...</b><br>"
                popup_html += f"<small style='color: #34495e;'>{res['snippet'][:100]}...</small><br>"
                popup_html += f"<a href='{res['link']}' target='_blank' style='color: #3498db;'>查看連結</a>"
                popup_html += "</div>"

            popup_html += "</div>"

            # 加上標記
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=350),
                tooltip=f"{location} ({len(results)} 筆結果)",
                icon=folium.Icon(color=colors[i % len(colors)], icon='info-sign')
            ).add_to(m)

    return m


def display_results_summary(analyzed_results):
    """
    顯示分析結果摘要
    """
    print("\n" + "="*80)
    print(f"找到 {len(analyzed_results)} 筆包含地理位置的搜尋結果")
    print("="*80 + "\n")

    for i, result in enumerate(analyzed_results, 1):
        print(f"【結果 {i}】")
        print(f"標題: {result['title']}")
        print(f"地點: {', '.join(result['locations'])}")
        print(f"描述: {result['snippet'][:100]}...")
        print(f"網址: {result['link']}")
        print("-"*80 + "\n")

## 5.搜尋並標註地圖

In [ ]:
def search_and_map(query, api_key, num_results=10):
    """
    整合搜尋與地圖標註的主函數

    參數:
        query (str): 搜尋關鍵字
        api_key (str): SerpAPI 金鑰
        num_results (int): 搜尋結果數量

    回傳:
        folium.Map: 標註好的地圖
    """
    print(f"正在搜尋: {query}\n")

    # 1. 執行搜尋
    results = search_google(query, api_key, num=num_results)

    if not results:
        print("沒有找到任何結果")
        return None

    print(f"✓ 找到 {len(results)} 筆搜尋結果")

    # 2. 分析結果並擷取地理位置
    analyzed_results = analyze_results_with_locations(results)

    if not analyzed_results:
        print("\n⚠️ 搜尋結果中沒有找到可辨識的台灣地區")
        return None

    print(f"✓ 從結果中找到 {len(analyzed_results)} 筆包含地理位置的資料\n")

    # 3. 顯示結果摘要
    display_results_summary(analyzed_results)

    # 4. 建立地圖
    print("正在建立地圖...\n")
    map_obj = create_map_from_results(analyzed_results)
    print("✓ 地圖建立完成！")

    return map_obj

## 6. 使用範例

In [ ]:
#搜尋台灣美食
query = "台灣必吃美食推薦"
map_result = search_and_map(query, SERPAPI_KEY, num_results=10)

if map_result:
    display(map_result)

正在搜尋: 台灣必吃美食推薦

✓ 找到 9 筆搜尋結果
✓ 從結果中找到 2 筆包含地理位置的資料


找到 2 筆包含地理位置的搜尋結果

【結果 1】
標題: 臺北飲食攻略
地點: 臺北
描述: 小籠包 ... 一層層巧手摺疊的麵衣，包裹著肉質鮮嫩的內餡，大口咬下，鮮甜的金黃湯汁在嘴裡流竄，這道源於江南的著名麵點，在臺灣發揚光大，是許多國際觀光客來臺指名必吃的美食。...
網址: https://www.travel.taipei/zh-tw/must-visit/tasterguide
--------------------------------------------------------------------------------

【結果 2】
標題: 【台灣美食】從南吃到北！全台必吃美食大集合
地點: 台北, 基隆
描述: 台灣美食 · 基隆美食不只廟口？！必吃排骨飯、魷魚羹、龍蝦麻糬別錯過 基隆是許多台北人週末出門的好地方，不管是想欣賞海景還是大啖美食，基隆通通都能滿足。 · 【基隆名產】在 ......
網址: https://www.funtime.com.tw/theme/taiwan-food
--------------------------------------------------------------------------------

正在建立地圖...

✓ 地圖建立完成！


In [ ]:
#搜尋台灣景點
query = "台灣熱門觀光景點"
map_result = search_and_map(query, SERPAPI_KEY, num_results=10)

if map_result:
    display(map_result)

正在搜尋: 台灣熱門觀光景點

✓ 找到 9 筆搜尋結果
✓ 從結果中找到 7 筆包含地理位置的資料


找到 7 筆包含地理位置的搜尋結果

【結果 1】
標題: 觀光景點
地點: 宜蘭, 日月潭
描述: 北起的東北角、北海岸及宜蘭地區，擁有各類特殊海岸地形，沿途山青水秀，天藍海深；往中南部地區，日月潭的湖光山色、八卦山賞鷹、水果之鄉的梨山、佛教聖地的獅頭山等，都可充分 ......
網址: https://www.taiwan.net.tw/m1.aspx?sNo=0001016
--------------------------------------------------------------------------------

【結果 2】
標題: 台灣熱門旅遊景點
地點: 高雄, 彰化, 南投
描述: 「橫山賞鷹平台」位於南投市永興里與彰化縣社頭鄉交界處，地勢高，可眺望彰化平原... 七坑溫泉位於高雄市桃源區，想要來此體驗泡湯樂趣的遊客，需由寶來溫泉區依指標行......
網址: https://okgo.tw/buty/
--------------------------------------------------------------------------------

【結果 3】
標題: 2025台灣景點一日遊懶人包，國內旅遊
地點: 桃園
描述: 2025推薦超過60個「桃園景點」推薦懶人包，從觀光工廠玩到森林瀑布一日遊，桃園市除了到拉拉山玩之外，千萬千萬不要忘了安排桃園景點一日遊，直接整理推薦超過 ......
網址: https://bunnyann.tw/attractions/
--------------------------------------------------------------------------------

【結果 4】
標題: 2025 年台灣絕美秘境：嚴選40 個從北到南的必去推薦景點
地點: 台北, 北投
描述: 推薦【台北必去秘境】圓山大飯店密道 · 推薦【台北內湖必去秘境】大溝溪生態治水園區 · 推薦【台北公館必去秘境】台電加羅林魚木 · 推薦【台北北投必去秘境】 ......
網址: https://hshsharehouse.com/taiwan-hidden-gems-4

In [ ]:
#自訂搜尋
custom_query = input("請輸入搜尋關鍵字 (包含地點相關): ")
num_results = int(input("幾筆結果? (5-20): "))

map_result = search_and_map(custom_query, SERPAPI_KEY, num_results=num_results)

if map_result:
    display(map_result)

請輸入搜尋關鍵字 (包含地點相關): 宜蘭
幾筆結果? (5-20): 8
正在搜尋: 宜蘭

✓ 找到 7 筆搜尋結果
✓ 從結果中找到 7 筆包含地理位置的資料


找到 7 筆包含地理位置的搜尋結果

【結果 1】
標題: 宜蘭縣- 維基百科，自由的百科全書
地點: 新北, 桃園, 臺中, 新竹, 宜蘭, 花蓮
描述: 宜蘭縣是中華民國臺灣省的縣，位於臺灣東部與北部，西北部邊界鄰近新北市，西鄰桃園市與新竹縣，西南鄰臺中市，南鄰花蓮縣，東臨太平洋。縣治設在宜蘭市，宜蘭縣內轄有1市、3 ......
網址: https://zh.wikipedia.org/zh-tw/%E5%AE%9C%E8%98%AD%E7%B8%A3
--------------------------------------------------------------------------------

【結果 2】
標題: 宜蘭縣政府全球資訊網- Yilan County Government
地點: 宜蘭, Yilan
描述: 代理縣長介紹 · 秘書長介紹 · 各局處首長及連絡電話 · 縣政藍圖 · 組織架構 · 宜蘭縣政府平面圖 · 首長新聞 · 看見好宜蘭 · 宜蘭好山水 · 我是宜蘭人 · 宜蘭特色諺語 ......
網址: https://www.e-land.gov.tw/Default.aspx
--------------------------------------------------------------------------------

【結果 3】
標題: 宜蘭縣
地點: 臺北, 宜蘭
描述: 宜蘭縣. 樂活宜蘭．幸福城市位處臺灣東北角的宜蘭縣，由於三面背山、一面向海特殊地形，孕育獨特文化與人情味，呈現以三生共構的世外桃源，如今藉由雪山隧道，拉近與臺北不到50 ......
網址: https://www.taiwan.net.tw/m1.aspx?sno=0001106
--------------------------------------------------------------------------------

【結果 4】
標題: 震撼空拍畫面曝！宜蘭冬山鄉「汪洋一片」晨曦照映積水 ...
地點: 宜蘭
描述: 受颱風「鳳

## 7. 儲存地圖(供參考，詳細請照作業需求)


In [ ]:
# 將地圖儲存為 HTML 檔案
if map_result:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"search_map_{timestamp}.html"
    map_result.save(filename)
    print(f"\n✓ 地圖已儲存為: {filename}")
    print("可以用瀏覽器開啟檢視！")


✓ 地圖已儲存為: search_map_20251113_035829.html
可以用瀏覽器開啟檢視！


In [ ]:
!pip install gradio -q

In [ ]:
import gradio as gr
import os

In [ ]:
def gradio_search_and_map(query: str, num_results: int):
    """
    Gradio 介面函數，用於執行搜尋並顯示地圖。
    """
    if not query:
        return "請輸入搜尋關鍵字。"
    if not (5 <= num_results <= 20):
        return "結果數量必須在 5 到 20 之間。"

    print(f"Gradio 搜尋觸發：{query}, 結果數：{num_results}")
    map_obj = search_and_map(query, SERPAPI_KEY, num_results=num_results)

    if map_obj:
        # 儲存地圖到臨時 HTML 檔案
        temp_filename = f"temp_map_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
        map_obj.save(temp_filename)
        print(f"地圖已暫存為：{temp_filename}")

        # Gradio 可以直接顯示 HTML 文件
        # return f"<iframe src='file/{temp_filename}' width='100%' height='500px'></iframe>"
        # 或者是直接讀取 HTML 內容返回
        with open(temp_filename, 'r', encoding='utf-8') as f:
            html_content = f.read()
        os.remove(temp_filename) # 處理完畢後刪除臨時檔案
        return html_content
    else:
        return "找不到相關地圖結果。"


In [ ]:
if __name__ == '__main__':
    # 建立 Gradio 介面
    interface = gr.Interface(
        fn=gradio_search_and_map,
        inputs=[
            gr.Textbox(label="請輸入搜尋關鍵字 (包含地點相關): ", placeholder="例如：台灣必吃美食推薦", value="台灣熱門觀光景點"),
            gr.Slider(minimum=5, maximum=20, value=10, step=1, label="選擇搜尋結果數量 (5-20):")
        ],
        outputs=gr.HTML(label="地圖結果"),
        title="台灣旅遊地圖搜尋器",
        description="""輸入關鍵字，搜尋相關的台灣地點並在地圖上標註。
        如果搜尋結果包含地理資訊，地圖將會顯示在下方；否則將會提示沒有相關地圖結果."""
    )

    # 啟動 Gradio 應用程式
    interface.launch(debug=True, share=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Gradio 搜尋觸發：台灣熱門觀光景點, 結果數：9
正在搜尋: 台灣熱門觀光景點

✓ 找到 9 筆搜尋結果
✓ 從結果中找到 7 筆包含地理位置的資料


找到 7 筆包含地理位置的搜尋結果

【結果 1】
標題: 觀光景點
地點: 宜蘭, 日月潭
描述: 北起的東北角、北海岸及宜蘭地區，擁有各類特殊海岸地形，沿途山青水秀，天藍海深；往中南部地區，日月潭的湖光山色、八卦山賞鷹、水果之鄉的梨山、佛教聖地的獅頭山等，都可充分 ......
網址: https://www.taiwan.net.tw/m1.aspx?sNo=0001016
--------------------------------------------------------------------------------

【結果 2】
標題: 台灣熱門旅遊景點
地點: 高雄, 彰化, 南投
描述: 「橫山賞鷹平台」位於南投市永興里與彰化縣社頭鄉交界處，地勢高，可眺望彰化平原... 七坑溫泉位於高雄市桃源區，想要來此體驗泡湯樂趣的遊客，需由寶來溫泉區依指標行......
網址: https://okgo.tw/buty/
--------------------------------------------------------------------------------

【結果 3】
標題: 2025台灣景點一日遊懶人包，國內旅遊
地點: 桃園
描述: 2025推薦超過60個「桃園景點」推薦懶人包，從觀光工廠玩到森林瀑布一日遊，桃園市除了到拉拉山玩之外，千萬千萬不要忘了安排桃園景點一日遊，直接整理推薦超過 ......
網址: https://bunnyann.tw/attractions/
--------------------------------------------------------------------------------

【結果 4】
標題: 2025 年台灣絕美秘境：嚴選40 個從北到南的必去推薦景點
地點: 台北, 北投
描述: 推薦【台北必去秘境】圓山大飯店密道 · 推薦【台北內湖必去秘境】大溝溪生態治水園區 · 推薦【台北公館必去秘境】台電加羅林魚木 · 推薦【台北北投必去秘境】 ......
網址: https://hshshareho